## <h1 id="Librer%C3%ADas-y-variables-de-entorno">Inicializacion del Entorno<a class="anchor-link" href="#Librer%C3%ADas-y-variables-de-entorno">¶</a></h1>

In [ ]:
!pip install vit_keras

In [ ]:
# UOC
# Máster de Ciencia de Datos 
# TFM - Refinamientos de modelos de IA para la identificación de cáncer de piel
# Manel Muñiz Peralvarez

import numpy as np
import pandas as pd
import os
import cv2
import sys
import timeit
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import asarray 
from datasets import load_dataset 
from sklearn import model_selection
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix
from vit_keras import vit
from tensorflow.keras.applications import EfficientNetB0, Xception, ResNet50, ResNet152, VGG16, DenseNet201, MobileNetV2, InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import warnings
from PIL import Image
import csv
import random

warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# GPU CUDA
# Directorio con las librerías CUDA
#os.add_dll_directory('C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.2\\bin')
# Obtención de información de la GPU dispponible
from tensorflow.python.client import device_lib
def print_info():
    print('  Versión de TensorFlow: {}'.format(tf.__version__))
    print('  GPU: {}'.format([x.physical_device_desc for x in device_lib.list_local_devices() if x.device_type=='GPU']))
    print('  Versión Cuda  -> {}'.format(tf.sysconfig.get_build_info()['cuda_version']))
    print('  Versión Cudnn -> {}\n'.format(tf.sysconfig.get_build_info()['cudnn_version']))
print_info()

In [ ]:
path_or = '/kaggle/input/datostfb/Datos/'

# De todas las configuraciones realizadas en el trabajo anterior, usamos CF2 la cuál es la de salida binaria benigno/maligno
CF = 'CF2'

path = 'D:/TFM/Codigo/Datos/CF2/'
path_img = 'D:/TFM/Codigo/Datos/CF2/imagenes/'
output_path = 'D:/TFM/Codigo/Datos/CF2/salidas/'

# Dimensiones de las imágenes para redes preentrenadas
WIDTH_224 = 224
HEIGHT_224 = 224

In [ ]:
# FLAGS 
CF1_flag = False
CF2_flag = True
CF3_flag = False
CF4_flag = False

# Aquí seleccionaremos como "True" los modelos que queramos entrenar. Por defecto se entrenan todos
train_model_EfficientNet_flag = True
test_model_EfficientNet_flag = True

train_model_xception_flag = True
test_model_xception_flag = True

train_model_resnet_flag = True
test_model_resnet_flag = True

train_model_VGG16_flag = True
test_model_VGG16_flag = True

train_model_densenet_flag = True
test_model_densenet_flag = True

train_model_mobilenet_flag = True
test_model_mobilenet_flag = True

train_model_InceptionResNetV2_flag = True
test_model_InceptionResNetV2_flag = True

save_matriz_pesos_flag = True

# Numero de variables en el dataset de metadatos
caract_metadata = 3

In [ ]:
if CF1_flag or CF4_flag:
    # Diccionario con las classes CF1
    classes = {0: ('bcc', 'CARCINOMA'),
               1: ('df', 'DERMATOFIBROMA'),
               2: ('bkl', 'LENTIGO'),
               3: ('vasc', 'LESION_VASCULAR'),
               4: ('mel', 'MELANOMA'),
               5: ('nv', 'NEVUS'),
               6: ('akiec', 'QUERATOSIS')}

    cm_plot_classes = ['bcc', 'df', 'bkl', 'vasc', 'mel', 'nv', 'akiec']

if CF2_flag or CF3_flag:
    # Diccionario con las classes CF2
    classes = {0: ('ben', 'BENIGNO'),
               1: ('mal', 'MALIGNO')}

    cm_plot_classes = ['ben', 'mal']

<h1 id="An%C3%A1lisis-exploratoria-de-datos">Análisis exploratoria de datos<a class="anchor-link" href="#An%C3%A1lisis-exploratoria-de-datos">¶</a></h1>

<h1 id="Funciones">Funciones<a class="anchor-link" href="#Funciones">¶</a></h1>

In [ ]:
def array_normalizar_conjunto_imagenes(x, conjunto_nuevo):
    for imagen in x:
        # Convertir la imagen a un arreglo NumPy
        img_array = np.array(imagen)
        conjunto_nuevo.append(img_array)
    conjunto_nuevo = 2 * (conjunto_nuevo - np.min(conjunto_nuevo)) / (np.max(conjunto_nuevo) - np.min(conjunto_nuevo)) - 1

In [ ]:
def modificar_resolucion_imagenes(x, conjunto_nuevo, anchura, altura):
    resolucion = (anchura, altura)
    for imagen in x:
        imagen_pil = Image.open(path_img+imagen)
        imagen2 = imagen_pil.convert("RGB")
        imagen_redimensionada = imagen2.resize(resolucion)
        conjunto_nuevo.append(imagen_redimensionada)

In [ ]:
# Función para la representación gráfica deL accuracy y loss de los conjuntos de entrenamiento y validación
def plot_acc_loss(histfit):
    """
    Función que representa las gráficas de accuracy y loss de los conjuntos de entrenamiento y validación.
    
    Parámetro de entrada:
    histfit: histórico del entrenamiento y validación
    """

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 4)) # 2 gráficas en la misma fila 
    fig.suptitle('Accuracy y Loss del modelo')
    sns.set_theme(style = 'darkgrid')
    
    # accuracy
    ax1.plot(histfit['accuracy'])
    ax1.plot(histfit['val_accuracy'])
    ax1.set(xlabel = 'Epoch', ylabel = 'Accuracy')
    ax1.legend(['train', 'valid'], loc = 'lower right')
       
    # loss
    ax2.plot(histfit['loss'])
    ax2.plot(histfit['val_loss'])
    ax2.set(xlabel = 'Epoch', ylabel = 'Loss')
    ax2.legend(['train','valid'], loc = "upper right")

In [ ]:
# Función para representar la matriz de confusión
def plot_confusion_matrix(cm, classes, normalize = False, cmap = plt.cm.Blues):
    """
    Función que representa la matriz de confusión.
    Se puede obtener normalizada en porcentajes con: normalize = True.
    
    Parámetros de entrada:
    cm: matriz de confusión
    classes: array con las clases
    normalize: boleano para indicar si se quieren los datos normalizados o no
    cmap: paleta de colores
    
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        title = 'Matriz de confusión normalizada en porcentajes'
    else:
        title = 'Matriz de confusión sin normalizar'
        
    plt.figure(figsize = (10,8))
    plt.title(title, size = 18)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45, size = 13)
    plt.yticks(tick_marks, classes, size = 13)
    fmt = '.2%' if normalize else 'd'
    cm_df = pd.DataFrame(cm,
                     index = classes, 
                     columns = classes)
    sns.heatmap(cm_df, annot = True, fmt = fmt, cmap = cmap)
    plt.ylabel('Realidad', size = 15)
    plt.xlabel('Predicción', size = 15)
    plt.tight_layout()

In [ ]:
# Función para representar la curva ROC multiclase
def plot_multiclass_roc(model, X_test, y_test, classes, n_classes, figsize = (14, 7)):
    """
    Función que devuelve una representación gráfica de la curva ROC multiclase.
    
    Parámetros de entrada:
    model: modelo con el que se realiza la predicción
    X_test: características del conjunto de test
    y_test: etiquetas del conjunto de test
    n_classes: entero con el número de clases
    figsize: dimensiones del gráfico
    """
    
    # Predicción
    y_pred = model.predict(X_test)
    
    # Diccionarios
    fpr = dict()     # ratio de FP
    tpr = dict()     # ratio de TP
    roc_auc = dict() # curva ROC
    
    # Codificación one hot
    y_test_dummies = pd.get_dummies(y_test, drop_first = False).values
    
    # Iteración por todas las clases
    for i in range(n_classes):
        fpr[i], tpr[i], _ = metrics.roc_curve(y_test_dummies[:, i], y_pred[:, i])
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])
        
    # ROC para cada clase
    fig, ax = plt.subplots(figsize = figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('Ratio de falsos positivos')
    ax.set_ylabel('Ratio de verdaderos positivos')
    ax.set_title('CURVA ROC')
    if CF1_flag or CF4_flag:
        for i in range(n_classes):
            ax.plot(fpr[i], tpr[i], label = 'Curva ROC (area = %0.2f) para la lesión cutánea %s' % (roc_auc[i], classes[i][1]))
    if CF2_flag or CF3_flag:
        ax.plot(fpr[0], tpr[0], label = 'Curva ROC (area = %0.2f) para la lesión cutánea %s' % (roc_auc[0], classes[0][1]))
        ax.plot(fpr[1], tpr[1], label = 'Curva ROC (area = %0.2f) para la lesión cutánea %s' % (roc_auc[1], classes[1][1]))
    ax.legend(loc = "best")
    ax.grid(alpha = .4)
    sns.despine()
    plt.show()

In [ ]:
# Función que entrena, valida y guarda los resultados en el diccionario histfit
def fn_train_val(histfit, model, x, y, batch_size, epochs, callbacks, x_val, y_val):
    """
    Función que entrena y valida el modelo con el conjunto de datos, número de épocas y tamaño del batch introducidos
    como parámetros de entrada.

    histfit: diccionario para guardar los resultados del entrenamiento y validación
    model: modelo a entrenar
    x: conjunto de entrenamiento
    y: etiquetas del conjunto de entrenamiento
    batch_size: tamaño del batch
    epochs: número de épocas   
    callbacks: devolución de llamadas
    x_val: 
    y_val = 

    
    Devuelve:
    histfit -- diccionario con los resultados del entrenamiento y validación
    """
    
    history = model.fit(x = x, y = y, batch_size = batch_size, epochs = epochs,
                            callbacks = callbacks, validation_data=(x_val, y_val))

    # Adición de los resultados al diccionario
    for key, value in history.history.items():
        for i in range(len(value)):
            if key != 'lr':
                histfit[key].append(value[i])
    return histfit

In [ ]:
# Función que entrena, valida y guarda los resultados en el diccionario histfit
def fn_train_val2(histfit, model, x_rgb, x_metadata, y, val_data, batch_size, epochs, callbacks, steps_per_epoch = None, validation_steps = None, validation_batch_size = None):
    """
    Función que entrena y valida el modelo con el conjunto de datos, número de épocas y tamaño del batch introducidos
    como parámetros de entrada.

    histfit: diccionario para guardar los resultados del entrenamiento y validación
    model: modelo a entrenar
    x: conjunto de entrenamiento
    y: etiquetas del conjunto de entrenamiento
    batch_size: tamaño del batch
    epochs: número de épocas   
    callbacks: devolución de llamadas
    steps_per_epoch: pasos por época
    validation_steps: pasos en la validación
    validation_batch_size: tamaño del batch en la validación

    
    Devuelve:
    histfit -- diccionario con los resultados del entrenamiento y validación
    """
    
    history = model.fit(x = [x_rgb,x_metadata], y = y, validation_data = val_data, batch_size = batch_size,
                            epochs = epochs, callbacks = callbacks, steps_per_epoch = steps_per_epoch, 
                            validation_steps = validation_steps, validation_batch_size = validation_batch_size, class_weight = class_weights )
    
    
    
    # Adición de los resultados al diccionario
    for key, value in history.history.items():
        for i in range(len(value)):
            if key != 'lr':
                histfit[key].append(value[i])
    return histfit

In [ ]:
# Función de clasificación de una imagen con un modelo determinado
def class_img(model_key, image):
    """
    Función que clasifica la imagen introducida con el modelo seleccionado.
    
    Parámetros de entrada:
    model_key: clave del modelo. Diccionario: saved_models.
    image: nombre del fichero que contiene la imagen a clasificar.
    
    Devuelve la clase predicha y el porcentaje de acierto de la clasificación.
    """
    
    # Asignación del modelo
    model = saved_models[model_key]
        
    # Ruta completa de la imagen a clasificar
    img_path = image
        
    if model_key == 1:
        width = WIDTH
        height = HEIGHT
    else:
        width = WIDTH_224
        height = HEIGHT_224
        
    # Obtención de la matriz de características de la imagen
    img_arr = get_array_from_img(img_path, width, height)
    # Dimensionamiento del conjunto de datos a las dimensiones esperada por el modelo
    x = np.array(img_arr).reshape(-1, width, height, 3)
            
    # Normalización
    x = (x - np.mean(x)) / np.std(x)
        
    # Clasificación de la imagen
    result = model.predict(x)
    class_pred = max(result[0])                                        # Clase predicha
    class_ind = list(result[0]).index(class_pred)                      # Índice de la clase predicha
    class_name = classes[class_ind][0] + ' · ' + classes[class_ind][1] # Descriptivo de la clase predicha
    print('***** CLASIFICANDO CON MODELO ' + str(model_key) + '/' + str(len(saved_models)) + ' *****\n')
    print('\tResultado de la clasificación: ' + class_name)
    print('\n\tCerteza del resultado: ' + str(round(class_pred * 100, 3)) + '%\n')
    
    return class_name, round(class_pred * 100, 3)

In [ ]:
# Función para la obtención de la matriz de características de una imagen
def get_array_from_img (img_path, width, height):
    """
    Función que lee la imagen introducida como parámetro de entrada y la reescala a las dimensiones introducidas.
    Devuelve una matriz de dimensiones width x height x 3 (RGB) con las características de la imagen.
    
    Parámetros de entrada:
    img_path: ruta completa (incluido el nombre del fichero) de la imagen de entrada
    width: ancho de la imagen de salida
    height: alto de la imagen de salida
    
    Devuelve:
    img_resized: matriz con las características de la imagen de entrada con dimensiones width x height x 3
    """
    try:
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)        # lectura y conversión de formato BGR a RGB
        dim = (width, height)
        img_resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) # reescalado de la imagen
    except Exception as e:
        print(e)
    return img_resized

In [ ]:
# Función que devuelve una matriz con las características y etiquetas de las imágenes con las dimensiones introducidas
def get_array_resized_images (width, height, range_ini, range_fin, path_img):
    """
    Función que obtiene la matriz de características de las imágenes con las dimensiones introducidas.
    
    Parámetros de entrada:
    width: ancho de la imagen de salida
    height: alto de la imagen de salida
    range_ini: nº de fila en la que empieza la selección
    range_fin: nº de fila (no incluida) en la que termina la selección  
    
    Devuelve:
    df_images_resized: matriz con las características de las imagénes con dimensiones:
    (range_fin-range_ini) x ((width x height x 3) + 1 de la etiqueta correspondiente)
    
    """
    # Array con los nombres de columnas de las dimensiones introducidas 
    columns = []
    for i in range(width * height * 3):
        columns.append('col_' + str(i))
    columns.append('label')
    
    # Matriz para guardar las características y etiquetas de todas las imágenes
    resized_images = []
    
    # Iteración de la tabla con los nombres de las imágenes
    for index, row in metadata.iloc[range_ini:range_fin].iterrows():
        # Obtención de la imagen matriz de características de la imagen en las nuevas dimensiones
        img_arr = get_array_from_img(path_img + row['image'], width, height)
        # Aplanado de la matriz a 1 dimensión
        flattened_img = img_arr.flatten()
        # Adición de la etiqueta de la imagen
        flattened_img = np.append(flattened_img, row['label'])
        # Adición del vector con las características de la imagen y etiqueta a la matriz con el resto de imágenes
        resized_images.append(flattened_img)
    
    # Conversión a dataframe
    df_resized_images = pd.DataFrame(resized_images, columns = columns) 
    
    return df_resized_images

In [ ]:
# Función con la configuración de callbacks
def def_callbacks(e_monitor = 'val_loss', e_mode = 'min', e_patience = 5, e_verbose = 2,
                  r_monitor = 'val_accuracy', r_factor = 0.2, r_patience = 2, r_min_delta = 1e-4,
                  r_min_lr = 1e-6, r_mode = 'max', r_verbose = 2, c_filepath = 'best_model.h5', 
                  c_monitor = 'val_accuracy', c_save_best_only = True, c_save_weights_only = True,
                  c_mode = 'max', c_verbose = 2):
    """
    Función que devuelve una lista con las opciones de callbacks
    
    Devuelve:
    callbacks: lista con la configuración de la parada temprana, caida de la tasa de aprendizaje y punto de control
    
    """
    
    # Parada temprana en el caso de que la pérdida en la validación no mejore en 5 épocas
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = e_monitor,
                                                      mode = e_mode,
                                                      patience = e_patience,
                                                      verbose = e_verbose)
    
    # Caida de la tasa de aprendizaje después de 2 épocas sin mejora en el accuracy de la validación
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = r_monitor,
                                                     factor = r_factor,
                                                     patience = r_patience,
                                                     min_delta = r_min_delta,
                                                     min_lr = r_min_lr,
                                                     mode = r_mode,
                                                     verbose = r_verbose)
    
    # Guardado de los pesos del mejor modelo en función del mejor accuracy en la validación
    checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = c_filepath,
                                                      monitor = c_monitor,
                                                      save_best_only = c_save_best_only,
                                                      save_weights_only = c_save_weights_only,
                                                      mode = c_mode,
                                                      verbose = c_verbose)
    
    # Array con los callbacks
    callbacks = [early_stopping, checkpointer, reduce_lr]
    
    return callbacks

In [ ]:
def ic_class_image(image):
    
    """
    Clasifica una imagen con la Inteligencia Colectiva (IC).
    
    Parámetros:
    image -- ruta completa de la imagen con el nombre del archivo que contiene la imagen.
    
    Devuelve:
    class_id -- identificador numérico de la lesión (0-6)
    lesion_skin_cod -- código de la lesión (akiec · bcc · bkl · df · mel · nv · vasc)
    lesion_skin_desc -- descripción larga de la lesión
    ic_accuracy -- porcentaje de exactitud del resultado
    ic_prob -- vector con las probabilidades para cada una de las 7 lesiones
    """
    
    # Matriz para guardar los resultados de las predicciones de cada modelo para cada clase
    if CF1_flag or CF4_flag:
        m2 = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64)

    if CF2_flag or CF3_flag:
        m2 = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
    
    # Vector para guardar los resultados de la IC
    ic_result = []
    # Vector para guardar la probalidad de cada clase de la IC
    ic_prob = []
    
    # Vector para guardar el número de redes que han predicho la clase predichca por la IC
    counted_pred = []
    
    # Predicción con cada modelo
    for key in saved_models.keys():
        model = saved_models[key]
        if key == 1:
            width = WIDTH
            height = HEIGHT
        else:
            width = WIDTH_224
            height = HEIGHT_224
        
        # Obtención de la matriz de características de la imagen
        img_arr = get_array_from_img(image, width, height)
        # Dimensionamiento del conjunto de datos a las dimensiones esperada por el modelo
        x = np.array(img_arr).reshape(-1, width, height, 3)
        # Normalización
        x = (x - np.mean(x)) / np.std(x)
        
        # Clasificación de la imagen con el modelo
        pred = model.predict(x, verbose=0)
        
        # Matriz temporal para guardar los resultados de las predicciones de este modelo
        if CF1_flag or CF4_flag:
            m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64)

        if CF2_flag or CF3_flag:
            m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
        
        # Adición de los resultados a la matriz con los resultados del resto de redes
        for i in range(len(classes)):
            m_temp['C'+str(i)] = [round(pred[0][i],5)]
            
        m2 = m2.append(m_temp)
        
    # Reinicio de los índices
    m2 = m2.reset_index()
    # Borrado de la columna index
    m2 = m2.drop(['index'], axis = 1)
    # Multiplicación de la matriz de pesos con la matriz de predicciones de la imagen
    m3 = m2.multiply(m1)
    print('Accuracy de cada modelo para cada clase (pesos generales)')
    print(m1)
    print('Accuracy de cada modelo para cada clase (predicciones de la imagen)')
    print(m2)
    print('Accuracy de cada modelo para cada clase (predicciones ponderadas de la imagen)')
    print(m3)
    # Guardado de los resultados de la inteligencia colectiva como la suma de valores de todas las redes
    for i in range(len(classes)):
        ic_result.append(m3['C' + str(i)].sum())
    
    # Probabilidad de cada clase
    for i in range(len(ic_result)):
        ic_prob.append(ic_result[i] / sum(ic_result))
    
    # Obtención del id de la clase predicha
    class_id = ic_result.index(max(ic_result))
    # Máximo marcador
    max_score = max(ic_result)
        
    # Búsqueda de la cantidad de redes que han predicho la clase predicha por la IC
    for i in range(len(saved_models)):
        counted_pred.append(m3.iloc[i].argmax())
        
    votes_pred = counted_pred.count(class_id)
    # Accuracy de la IC como la máxima puntuación entre las redes que han votado la clase predicha
    ic_accuracy = round(max_score / votes_pred, 2)
    # Código de la lesión cutánea
    lesion_skin_cod = classes[class_id][0]
    # Literal de la lesión cutánea    
    lesion_skin_desc = classes[class_id][1]  
            
    return class_id, lesion_skin_cod, lesion_skin_desc, ic_accuracy, ic_prob

<h1 id="Entrenamientos">Entrenamientos<a class="anchor-link" href="#Entrenamientos">¶</a></h1>

In [ ]:
# Matriz para guardar los pesos del accuracy de cada modelo para cada clase
if CF1_flag or CF4_flag:
    m1 = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 

if CF2_flag or CF3_flag:
    m1 = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)

In [ ]:
# Importación de los conjuntos de train y test
x_train = np.load(output_path + 'x_train.npy')
x_test = np.load(output_path + 'x_test.npy')
y_train = np.load(output_path + 'y_train.npy')
y_test = np.load(output_path + 'y_test.npy')
x_val = np.load(output_path + 'x_val.npy')
y_val = np.load(output_path + 'y_val.npy')

x_train_ext = np.load(output_path + 'x_train_ext.npy')
x_test_ext = np.load(output_path + 'x_test_ext.npy')
x_val_ext = np.load(output_path + 'x_val_ext.npy')

# Definimos la validation data para un modelo con multiples inputs
validation_data =  ([x_val,x_val_ext], y_val)

print("Forma de x_train:", x_train.shape)
print("Forma de x_train_ext:", x_train_ext.shape)
print(type(x_train))
print(type(x_train_ext))

In [ ]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)
y_val = y_val.astype(int)

<h1 id="Modelo-1-%C2%B7-CNN-adhoc">Modelo 1 · CNN adhoc<a class="anchor-link" href="#Modelo-1-%C2%B7-CNN-adhoc">¶</a></h1>

<h1 id="Modelo-2-%C2%B7-Red-transformer-ViT-B16">Modelo 2 · Red transformer ViT-B16<a class="anchor-link" href="#Modelo-2-%C2%B7-Red-transformer-ViT-B16">¶</a></h1>

<h1 id="Modelo-3-%C2%B7-Swin-Transformer">Modelo 3 · Swin Transformer<a class="anchor-link" href="#Modelo-3-%C2%B7-Swin-Transformer">¶</a></h1>

<h1 id="Modelo-4-%C2%B7-Red-EfficientNet-B0">Modelo 4 · Red EfficientNet B0<a class="anchor-link" href="#Modelo-4-%C2%B7-Red-EfficientNet-B0">¶</a></h1>

In [ ]:
def build_model_EfficientNet():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = EfficientNetB0(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                        # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_EfficientNet_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    # Define the input layer
    input_rgb = layers.Input(shape=(224, 224, 3), name='input_rgb')
    
    # Create the EfficientNet B0 model without the top classification layers
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_rgb)

    # Freeze the base model
    base_model.trainable = True

    # Get the output of the base model
    x = base_model(input_rgb)
    
    # Add additional layers to the base model output
    x = keras.layers.AveragePooling2D(pool_size = (2 ,2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(16, activation='relu')(x)
    
    input_metadata = layers.Input(shape=(caract_metadata,), name='input_metadata')
    x_final = keras.layers.Concatenate()([x, input_metadata])
    
    outputs = keras.layers.Dense(2, activation='softmax')(x_final)
    
    # Create the model
    model = models.Model(inputs=[input_rgb, input_metadata], outputs=outputs)
    
    return model

In [ ]:
if train_model_EfficientNet_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_EfficientNet()
    if CF2_flag or CF3_flag:
        model = build_model_EfficientNet_binary()
    model.summary()

In [ ]:
if train_model_EfficientNet_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32,64]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
print(class_weights)

class_weights = {0: .74, 1: 1.54}

In [ ]:
if train_model_EfficientNet_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history4 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo EfficientNet B0...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_EfficientNet()
                if CF2_flag or CF3_flag:
                    model = build_model_EfficientNet_binary()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                if CF1_flag or CF4_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
                if CF2_flag or CF3_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])    
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, x_train, x_train_ext, y_train, validation_data, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)                     
                # Adición de los resultados al diccionario history4
                history4['epochs'].append(epochs)
                history4['batch_size'].append(batch_size)
                history4['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history4['loss'].append(histfit['loss'][max_val_acc_idx])
                history4['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history4['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history4['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history4['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(output_path + 'df_histfit4')
                    model.save(output_path + 'efficientNet_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history4 = pd.DataFrame.from_dict(history4)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history4.to_pickle(output_path + 'df_history4')

In [ ]:
if test_model_EfficientNet_flag == True:
    # Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
    df_history4 = pd.read_pickle(output_path + 'df_history4')

    # Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
    df_history4.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
if test_model_EfficientNet_flag == True:
    # Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
    df_histfit4 = pd.read_pickle(output_path + 'df_histfit4')

    # Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
    plot_acc_loss(df_histfit4)

In [ ]:
if test_model_EfficientNet_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_EfficientNet()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_EfficientNet_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'efficientNet_best_model.h5')
    model.save(output_path + 'model4.h5') # Guardado completo del modelo

    loss, acc = model.evaluate([x_test,x_test_ext], y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
if test_model_EfficientNet_flag == True:
    # Carga del modelo completo
    if CF1_flag or CF4_flag:
        model = build_model_EfficientNet()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_EfficientNet_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'model4.h5')

In [ ]:
if test_model_EfficientNet_flag == True:
    # Predicción con el conjunto de test
    y_pred = model.predict([x_test,x_test_ext])

In [ ]:
if test_model_EfficientNet_flag == True:
    # Curva ROC multiclase
    plot_multiclass_roc(model, [x_test,x_test_ext], y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
if test_model_EfficientNet_flag == True:
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

    # Cálculo de la matriz de confusión y representación normalizada y sin normalizar
    plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
    plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
if test_model_EfficientNet_flag == True:
    # Métricas
    print('***** MÉTRICAS *****')
    print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
    print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
if test_model_EfficientNet_flag == True:
    # Dataframe temporal para guardar los accuracies de cada clase de este modelo
    if CF1_flag or CF4_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
    if CF2_flag or CF3_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
    for i in range(len(classes)):
        m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]

    # Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
    m1 = pd.concat([m1, m_temp])

<h1 id="Modelo-5-%C2%B7-Red-Xception">Modelo 5 · Red Xception<a class="anchor-link" href="#Modelo-5-%C2%B7-Red-Xception">¶</a></h1>

In [ ]:
def build_model_Xception():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = Xception(include_top = False,                         # no incluir la capa de clasificación
                           input_tensor = inputs, 
                           weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_Xception_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Define the input layer
    input_rgb = layers.Input(shape=(224, 224, 3), name='input_rgb')
    
    # Create the Xception model without the top classification layers
    base_model = Xception(include_top=False, weights='imagenet', input_tensor=input_rgb)

    # Entrenar todas las capas, ajuste fino
    base_model.trainable = True

    # Get the output of the base model
    x = base_model(input_rgb)
    
    # Add additional layers to the base model output
    x = keras.layers.AveragePooling2D(pool_size = (2 ,2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(16, activation='relu')(x)
    
    input_metadata = layers.Input(shape=(caract_metadata,), name='input_metadata')
    x_final = keras.layers.Concatenate()([x, input_metadata])
    
    outputs = keras.layers.Dense(2, activation='softmax')(x_final)
    
    # Create the model
    model = models.Model(inputs=[input_rgb, input_metadata], outputs=outputs)
     
    return model

In [ ]:
if train_model_xception_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_Xception()
    if CF2_flag or CF3_flag:
        model = build_model_Xception_binary()
    model.summary()

In [ ]:
if train_model_xception_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32,64]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]

In [ ]:
if train_model_xception_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history5 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo Xception...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_Xception()
                if CF2_flag or CF3_flag:
                    model = build_model_Xception_binary()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                if CF1_flag or CF4_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
                if CF2_flag or CF3_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])       
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, x_train, x_train_ext, y_train, validation_data, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)           
                # Adición de los resultados al diccionario history5
                history5['epochs'].append(epochs)
                history5['batch_size'].append(batch_size)
                history5['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history5['loss'].append(histfit['loss'][max_val_acc_idx])
                history5['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history5['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history5['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history5['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(output_path + 'df_histfit5')
                    model.save(output_path + 'Xception_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history5 = pd.DataFrame.from_dict(history5)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history5.to_pickle(output_path + 'df_history5')

In [ ]:
if train_model_xception_flag == True:
    # Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
    df_history5 = pd.read_pickle(output_path + 'df_history5')

    # Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
    df_history5.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
if train_model_xception_flag == True:
    # Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
    df_histfit5 = pd.read_pickle(output_path + 'df_histfit5')

    # Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
    plot_acc_loss(df_histfit5)

In [ ]:
if test_model_xception_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_Xception()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_Xception_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'Xception_best_model.h5')
    model.save(output_path + 'model5.h5') # Guardado completo del modelo

    loss, acc = model.evaluate([x_test,x_test_ext], y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
if train_model_xception_flag == True:
    if CF1_flag or CF4_flag:
        model = build_model_Xception()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_Xception_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'model5.h5')

In [ ]:
if train_model_xception_flag == True:
    # Predicción con el conjunto de test
    y_pred = model.predict([x_test,x_test_ext])

In [ ]:
if train_model_xception_flag == True:
    # Curva ROC multiclase
    plot_multiclass_roc(model, [x_test,x_test_ext], y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
if train_model_xception_flag == True:
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))

    # Cálculo de la matriz de confusión y representación normalizada y sin normalizar
    plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
    plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
if train_model_xception_flag == True:
    # Métricas
    print('***** MÉTRICAS *****')
    print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
    print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
if train_model_xception_flag == True:
    # Dataframe temporal para guardar los accuracies de cada clase de este modelo
    if CF1_flag or CF4_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
    if CF2_flag or CF3_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
    for i in range(len(classes)):
        m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]

    # Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
    m1 = pd.concat([m1, m_temp])

<h1 id="Modelo-6-%C2%B7-ResNet152">Modelo 6 · ResNet152<a class="anchor-link" href="#Modelo-6-%C2%B7-ResNet152">¶</a></h1>

In [ ]:
def build_model_ResNet152():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = ResNet152(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(2, activation = 'softmax'),
  ])
 
    return model
def build_model_ResNet152_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Define the input layer
    input_rgb = layers.Input(shape=(224, 224, 3), name='input_rgb')
    
    # Create the ResNet152 model without the top classification layers
    base_model = ResNet152(include_top=False, weights='imagenet', input_tensor=input_rgb)

    # Entrenar todas las capas, ajuste fino
    base_model.trainable = True

    # Get the output of the base model
    x = base_model(input_rgb)
    
    # Add additional layers to the base model output
    x = keras.layers.AveragePooling2D(pool_size = (2 ,2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(16, activation='relu')(x)
    
    input_metadata = layers.Input(shape=(caract_metadata,), name='input_metadata')
    x_final = keras.layers.Concatenate()([x, input_metadata])
    
    outputs = keras.layers.Dense(2, activation='softmax')(x_final)
    
    # Create the model
    model = models.Model(inputs=[input_rgb, input_metadata], outputs=outputs)
     
    return model

In [ ]:
if train_model_resnet_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_ResNet152()
    if CF2_flag or CF3_flag:
        model = build_model_ResNet152_binary()
    model.summary()

In [ ]:
if train_model_resnet_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32]
    # learning rates
    learning_rates = [0.0005, 0.0001, 0.00005]

In [ ]:
if train_model_resnet_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history6 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo ResNet152...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_ResNet152()
                if CF2_flag or CF3_flag:
                    model = build_model_ResNet152_binary()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                if CF1_flag or CF4_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
                if CF2_flag or CF3_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])   
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, x_train, x_train_ext, y_train, validation_data, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)    

                # Adición de los resultados al diccionario history6
                history6['epochs'].append(epochs)
                history6['batch_size'].append(batch_size)
                history6['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history6['loss'].append(histfit['loss'][max_val_acc_idx])
                history6['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history6['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history6['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history6['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(output_path + 'df_histfit6')
                    model.save(output_path + 'ResNet152_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history6 = pd.DataFrame.from_dict(history6)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history6.to_pickle(output_path + 'df_history6')

In [ ]:
if train_model_resnet_flag == True:
    # Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
    df_history6 = pd.read_pickle(output_path + 'df_history6')
    
    # Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
    df_history6.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
if train_model_resnet_flag == True:
    # Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
    df_histfit6 = pd.read_pickle(output_path + 'df_histfit6')
    
    # Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
    plot_acc_loss(df_histfit6)

In [ ]:
if test_model_resnet_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_ResNet152()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_ResNet152_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'ResNet152_best_model.h5')
    #model.save(output_path + 'model6.h5') # guardado del modelo completo
    
    loss, acc = model.evaluate([x_test,x_test_ext], y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
if train_model_resnet_flag == True:
    # Carga del modelo completo
    if CF1_flag or CF4_flag:
        model = build_model_ResNet152()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_ResNet152_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'model6.h5')

In [ ]:
if train_model_resnet_flag == True:  
    # Predicción con el conjunto de test
    y_pred = model.predict([x_test,x_test_ext])

In [ ]:
if train_model_resnet_flag == True:
    # Curva ROC multiclase
    plot_multiclass_roc(model, [x_test,x_test_ext], y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
if train_model_resnet_flag == True:
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))
    
    # Cálculo de la matriz de confusión y representación normalizada y sin normalizar
    plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
    plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
if train_model_resnet_flag == True:
    # Métricas
    print('***** MÉTRICAS *****')
    print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
    print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
if train_model_resnet_flag == True:    
    # Dataframe temporal para guardar los accuracies de cada clase de este modelo
    if CF1_flag or CF4_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
    if CF2_flag or CF3_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
    for i in range(len(classes)):
        m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
        
    # Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
    m1 = pd.concat([m1, m_temp])

<h1 id="Model-7-%C2%B7-VGG16">Model 7 · VGG16<a class="anchor-link" href="#Model-7-%C2%B7-VGG16">¶</a></h1>

In [ ]:
def build_model_VGG16():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = VGG16(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_VGG16_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Define the input layer
    input_rgb = layers.Input(shape=(224, 224, 3), name='input_rgb')
    
    # Create the VGG16 model without the top classification layers
    base_model = VGG16(include_top=False, weights='imagenet', input_tensor=input_rgb)

    # Entrenar todas las capas, ajuste fino
    base_model.trainable = True

    # Get the output of the base model
    x = base_model(input_rgb)
    
    # Add additional layers to the base model output
    x = keras.layers.AveragePooling2D(pool_size = (2 ,2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(16, activation='relu')(x)
    
    input_metadata = layers.Input(shape=(caract_metadata,), name='input_metadata')
    x_final = keras.layers.Concatenate()([x, input_metadata])
    
    outputs = keras.layers.Dense(2, activation='softmax')(x_final)
    
    # Create the model
    model = models.Model(inputs=[input_rgb, input_metadata], outputs=outputs)
     
    return model

In [ ]:
if train_model_VGG16_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_VGG16()
    if CF2_flag or CF3_flag:
        model = build_model_VGG16_binary()
    model.summary()

In [ ]:
if train_model_VGG16_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32,64]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]

In [ ]:
if train_model_VGG16_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history7 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo VGG16...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_VGG16()
                if CF2_flag or CF3_flag:
                    model = build_model_VGG16_binary()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                if CF1_flag or CF4_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
                if CF2_flag or CF3_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])        
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, x_train, x_train_ext, y_train, validation_data, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)         
                # Adición de los resultados al diccionario history7
                history7['epochs'].append(epochs)
                history7['batch_size'].append(batch_size)
                history7['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history7['loss'].append(histfit['loss'][max_val_acc_idx])
                history7['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history7['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history7['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history7['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(output_path + 'df_histfit7')
                    model.save(output_path + 'VGG16_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history7 = pd.DataFrame.from_dict(history7)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history7.to_pickle(output_path + 'df_history7')

In [ ]:
if train_model_VGG16_flag == True:
    # Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
    df_history7 = pd.read_pickle(output_path + 'df_history7')
    
    # Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
    df_history7.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
if train_model_VGG16_flag == True:
    # Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
    df_histfit7 = pd.read_pickle(output_path + 'df_histfit7')
    
    # Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
    plot_acc_loss(df_histfit7)

In [ ]:
if test_model_VGG16_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_VGG16()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_VGG16_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'VGG16_best_model.h5')
    #model.save(output_path + 'model7.h5') # guardado del modelo completo

    loss, acc = model.evaluate([x_test,x_test_ext], y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
if train_model_VGG16_flag == True:
    # Carga del modelo completo
    if CF1_flag or CF4_flag:
        model = build_model_VGG16()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_VGG16_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'VGG16_best_model.h5')

In [ ]:
if train_model_VGG16_flag == True:
    # Predicción con el conjunto de test
    y_pred = model.predict([x_test,x_test_ext])

In [ ]:
if train_model_VGG16_flag == True:
    # Curva ROC multiclase
    plot_multiclass_roc(model, [x_test,x_test_ext], y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
if train_model_VGG16_flag == True:
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))
    
    # Cálculo de la matriz de confusión y representación normalizada y sin normalizar
    plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
    plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
if train_model_VGG16_flag == True:
    # Métricas
    print('***** MÉTRICAS *****')
    print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
    print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
if train_model_VGG16_flag == True:
    # Dataframe temporal para guardar los accuracies de cada clase de este modelo
    if CF1_flag or CF4_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
    if CF2_flag or CF3_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
    for i in range(len(classes)):
        m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
        
    # Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
    m1 = pd.concat([m1, m_temp])

<h1 id="Model-8-%C2%B7-DenseNet201">Model 8 · DenseNet201<a class="anchor-link" href="#Model-8-%C2%B7-DenseNet201">¶</a></h1>

In [ ]:
def build_model_DenseNet201():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = DenseNet201(include_top = False,                   # no incluir la capa de clasificación
                        input_tensor = inputs,
                        weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_DenseNet201_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Define the input layer
    input_rgb = layers.Input(shape=(224, 224, 3), name='input_rgb')
    
    # Create the DenseNet201 model without the top classification layers
    base_model = DenseNet201(include_top=False, weights='imagenet', input_tensor=input_rgb)

    # Entrenar todas las capas, ajuste fino
    base_model.trainable = True

    # Get the output of the base model
    x = base_model(input_rgb)
    
    # Add additional layers to the base model output
    x = keras.layers.AveragePooling2D(pool_size = (2 ,2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(16, activation='relu')(x)
    
    input_metadata = layers.Input(shape=(caract_metadata,), name='input_metadata')
    x_final = keras.layers.Concatenate()([x, input_metadata])
    
    outputs = keras.layers.Dense(2, activation='softmax')(x_final)
    
    # Create the model
    model = models.Model(inputs=[input_rgb, input_metadata], outputs=outputs)
 
    return model

In [ ]:
if train_model_densenet_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_DenseNet201()
    if CF2_flag or CF3_flag:
        model = build_model_DenseNet201_binary()
    model.summary()

In [ ]:
if train_model_densenet_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 12, 16]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]

In [ ]:
if train_model_densenet_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history8 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo DenseNet201...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_DenseNet201()
                if CF2_flag or CF3_flag:
                    model = build_model_DenseNet201_binary()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                if CF1_flag or CF4_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
                if CF2_flag or CF3_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])        
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, x_train, x_train_ext, y_train, validation_data, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)         
                # Adición de los resultados al diccionario history8
                history8['epochs'].append(epochs)
                history8['batch_size'].append(batch_size)
                history8['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history8['loss'].append(histfit['loss'][max_val_acc_idx])
                history8['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history8['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history8['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history8['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(output_path + 'df_histfit8')
                    model.save(output_path + 'DenseNet201_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history8 = pd.DataFrame.from_dict(history8)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history8.to_pickle(output_path + 'df_history8')


In [ ]:
if train_model_densenet_flag == True:
    # Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
    df_history8 = pd.read_pickle(output_path + 'df_history8')
    
    # Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
    df_history8.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
if train_model_densenet_flag == True:
    # Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
    df_histfit8 = pd.read_pickle(output_path + 'df_histfit8')
    
    # Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
    plot_acc_loss(df_histfit8)

In [ ]:
if test_model_densenet_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_DenseNet201()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_DenseNet201_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'DenseNet201_best_model.h5')
    #model.save(output_path + 'model8.h5') # guardado del modelo completo
    
    loss, acc = model.evaluate([x_test,x_test_ext], y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
if train_model_densenet_flag == True:
    # Carga del modelo completo
    if CF1_flag or CF4_flag:
        model = build_model_DenseNet201()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_DenseNet201_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'model8.h5')

In [ ]:
if train_model_densenet_flag == True:
    # Predicción con el conjunto de test
    y_pred = model.predict([x_test,x_test_ext], verbose=0)

In [ ]:
if train_model_densenet_flag == True:
    # Curva ROC multiclase
    plot_multiclass_roc(model, [x_test,x_test_ext], y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
if train_model_densenet_flag == True:
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))
    
    # Cálculo de la matriz de confusión y representación normalizada y sin normalizar
    plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
    plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
if train_model_densenet_flag == True:
    # Métricas
    print('***** MÉTRICAS *****')
    print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
    print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
if train_model_densenet_flag == True:
    # Dataframe temporal para guardar los accuracies de cada clase de este modelo
    if CF1_flag or CF4_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
    if CF2_flag or CF3_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
    for i in range(len(classes)):
        m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
        
    # Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
    m1 = pd.concat([m1, m_temp])

<h1 id="Model-9-%C2%B7-MobileNetV2">Model 9 · MobileNetV2<a class="anchor-link" href="#Model-9-%C2%B7-MobileNetV2">¶</a></h1>

In [ ]:
def build_model_MobileNetV2():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = MobileNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_MobileNetV2_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Define the input layer
    input_rgb = layers.Input(shape=(224, 224, 3), name='input_rgb')
    
    # Create the MobileNetV2 model without the top classification layers
    base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_rgb)

    # Entrenar todas las capas, ajuste fino
    base_model.trainable = True

    # Get the output of the base model
    x = base_model(input_rgb)
    
    # Add additional layers to the base model output
    x = keras.layers.AveragePooling2D(pool_size = (2 ,2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(16, activation='relu')(x)
    
    input_metadata = layers.Input(shape=(caract_metadata,), name='input_metadata')
    x_final = keras.layers.Concatenate()([x, input_metadata])
    
    outputs = keras.layers.Dense(2, activation='softmax')(x_final)
    
    # Create the model
    model = models.Model(inputs=[input_rgb, input_metadata], outputs=outputs)
 
    return model

In [ ]:
if train_model_mobilenet_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_MobileNetV2()
    if CF2_flag or CF3_flag:
        model = build_model_MobileNetV2_binary()
    model.summary()

In [ ]:
if train_model_mobilenet_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32,64]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]

In [ ]:
if train_model_mobilenet_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history9 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo MobileNetV2...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_MobileNetV2()
                if CF2_flag or CF3_flag:
                    model = build_model_MobileNetV2_binary()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                if CF1_flag or CF4_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
                if CF2_flag or CF3_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])       
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, x_train, x_train_ext, y_train, validation_data, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)           
                # Adición de los resultados al diccionario history9
                history9['epochs'].append(epochs)
                history9['batch_size'].append(batch_size)
                history9['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history9['loss'].append(histfit['loss'][max_val_acc_idx])
                history9['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history9['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history9['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history9['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(output_path + 'df_histfit9')
                    model.save(output_path + 'MobileNetV2_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history9 = pd.DataFrame.from_dict(history9)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history9.to_pickle(output_path + 'df_history9')

In [ ]:
if train_model_mobilenet_flag == True:
    # Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
    df_history9 = pd.read_pickle(output_path + 'df_history9')
    
    # Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
    df_history9.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
if train_model_mobilenet_flag == True:  
    # Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
    df_histfit9 = pd.read_pickle(output_path + 'df_histfit9')
    
    # Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
    plot_acc_loss(df_histfit9)

In [ ]:
if test_model_mobilenet_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_MobileNetV2()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_MobileNetV2_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'MobileNetV2_best_model.h5')
    #model.save(output_path + 'model9.h5') # guardado del modelo completo

    loss, acc = model.evaluate([x_test,x_test_ext], y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
if train_model_mobilenet_flag == True:
    # Carga del modelo completo
    if CF1_flag or CF4_flag:
        model = build_model_MobileNetV2()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_MobileNetV2_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'model9.h5')

In [ ]:
if train_model_mobilenet_flag == True:
    # Predicción con el conjunto de test
    y_pred = model.predict([x_test,x_test_ext])

In [ ]:
if train_model_mobilenet_flag == True:
    # Curva ROC multiclase
    plot_multiclass_roc(model, [x_test,x_test_ext], y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
if train_model_mobilenet_flag == True:  
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))
    
    # Cálculo de la matriz de confusión y representación normalizada y sin normalizar
    plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
    plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
if train_model_mobilenet_flag == True:
    # Métricas
    print('***** MÉTRICAS *****')
    print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
    print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
if train_model_mobilenet_flag == True:
    # Dataframe temporal para guardar los accuracies de cada clase de este modelo
    if CF1_flag or CF4_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
    if CF2_flag or CF3_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
    for i in range(len(classes)):
        m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
        
    # Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
    m1 = pd.concat([m1, m_temp])

<h1 id="Model-10-%C2%B7-InceptionResNetV2">Model 10 · InceptionResNetV2<a class="anchor-link" href="#Model-10-%C2%B7-InceptionResNetV2">¶</a></h1>

In [ ]:
def build_model_InceptionResNetV2():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 7 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    inputs = keras.layers.Input(shape = (224, 224, 3))
    model = InceptionResNetV2(include_top = False,                   # no incluir la capa de clasificación
                      input_tensor = inputs,
                      weights = 'imagenet')                  # preentrenado con el dataset imagenet.

    # Entrenar todas las capas, ajuste fino
    model.trainable = True
    
    model = Sequential([model, 
                      keras.layers.AveragePooling2D(pool_size = (2 ,2)),      
                      keras.layers.Flatten(), 
                      keras.layers.Dense(64, activation = 'relu'),
                      keras.layers.Dropout(0.2), 
                      keras.layers.Dense(16, activation = 'relu'),
                      keras.layers.Dense(7, activation = 'softmax'),
  ])
 
    return model
def build_model_InceptionResNetV2_binary():
    
    """
    Crea un modelo de red neuronal que utiliza la red EfficientNet B0, preentrenada con el dataset imagenet,
    al que se le sustituyen las capas top de clasificación por las siguientes:
    1 capa de pooling de media global 2D,
    1 capa de normalización del batch,
    1 capa del 20% de descartes,
    1 capa densa de salida con 2 neuronas y función de activación Softmax.
    
    Devuelve:
    model -- objecto de Keras que representa la red neuronal convolucional
    """
    
    # Define the input layer
    input_rgb = layers.Input(shape=(224, 224, 3), name='input_rgb')
    
    # Create the InceptionResNetV2 model without the top classification layers
    base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=input_rgb)

    # Entrenar todas las capas, ajuste fino
    base_model.trainable = True

    # Get the output of the base model
    x = base_model(input_rgb)
    
    # Add additional layers to the base model output
    x = keras.layers.AveragePooling2D(pool_size = (2 ,2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(16, activation='relu')(x)
    
    input_metadata = layers.Input(shape=(caract_metadata,), name='input_metadata')
    x_final = keras.layers.Concatenate()([x, input_metadata])
    
    outputs = keras.layers.Dense(2, activation='softmax')(x_final)
    
    # Create the model
    model = models.Model(inputs=[input_rgb, input_metadata], outputs=outputs)
 
    return model

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Mostrar estructura de la red
    if CF1_flag or CF4_flag:
        model = build_model_InceptionResNetV2()
    if CF2_flag or CF3_flag:
        model = build_model_InceptionResNetV2_binary()
    model.summary()

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Hiperparámetros
    # epochs
    n_epochs = [50]
    # batch sizes
    batch_sizes = [8, 16, 32,64]
    # learning rates
    learning_rates = [0.001, 0.0005, 0.0001]

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Diccionario para almacenar los resultados del entrenamiento
    history10 = {'epochs':[], 'batch_size':[], 'learning_rate':[],'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[], 'elapsed_time':[]}

    # Variable per guardar el mejor accuracy obtenido con el conjunto de validación
    best_val_acc = 0

    # Callbacks
    callbacks = def_callbacks()

    # Iterar por todas las combinaciones de hiperparámetros
    print('Entrenando el modelo InceptionResNetV2...')
    for epochs in n_epochs:
        for batch_size in batch_sizes:
            train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
            valid_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
            
            train_dataset = train_dataset.batch(batch_size)
            valid_dataset = valid_dataset.batch(batch_size)
            for learning_rate in learning_rates:
                # Diccionario para guardar los resultados de un entrenamiento
                histfit = {'loss':[], 'accuracy':[], 'val_loss':[], 'val_accuracy':[]}
                print('\tHiperparámetros: epochs: {} | batch_size: {} | learning_rate: {}'.format(epochs, batch_size, learning_rate))
                # Creación y compilación del modelo
                if CF1_flag or CF4_flag:
                    model = build_model_InceptionResNetV2()
                if CF2_flag or CF3_flag:
                    model = build_model_InceptionResNetV2_binary()
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                if CF1_flag or CF4_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
                if CF2_flag or CF3_flag:
                    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])       
                steps_per_epoch = np.ceil(len(x_train) / batch_size)
                validation_steps = np.ceil(len(x_test) / batch_size)
                validation_batch_size = batch_size
                # Con Data Augmentation
                validation_split = 0.0
                # Cómputo del tiempo transcurrido durante el entrenamiento y validación
                elapsed_time = timeit.timeit('fn_train_val2(histfit, model, x_train, x_train_ext, y_train, validation_data, batch_size, epochs,'+
                                             'callbacks)', globals = globals(), number = 1)            
                # Adición de los resultados al diccionario history10
                history10['epochs'].append(epochs)
                history10['batch_size'].append(batch_size)
                history10['learning_rate'].append(learning_rate)
                # Obtenemos el índice con el accuracy máximo con el conjunto de validación 
                max_val_acc_idx = histfit['val_accuracy'].index(max(histfit['val_accuracy']))
                history10['loss'].append(histfit['loss'][max_val_acc_idx])
                history10['accuracy'].append(histfit['accuracy'][max_val_acc_idx])
                history10['val_loss'].append(histfit['val_loss'][max_val_acc_idx])
                history10['val_accuracy'].append(histfit['val_accuracy'][max_val_acc_idx])
                history10['elapsed_time'].append(elapsed_time)

                # Guardamos el modelo con mejor accuracy en la validación
                if histfit['val_accuracy'][max_val_acc_idx] > best_val_acc:
                    best_val_acc = histfit['val_accuracy'][max_val_acc_idx]
                    df_histfit = pd.DataFrame.from_dict(histfit)
                    df_histfit.to_pickle(output_path + 'df_histfit10')
                    model.save(output_path + 'InceptionResNetV2_best_model.h5')

    # Conversión del dataframe a diccionario
    df_history10 = pd.DataFrame.from_dict(history10)

    # Guardado del dataframe con los resultados de todos los entrenamientos
    df_history10.to_pickle(output_path + 'df_history10')

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Recuperación del dataframe con los resultados de los entrenamientos con todas las combinaciones de hiperparámetros definidas
    df_history10 = pd.read_pickle(output_path + 'df_history10')
    
    # Obtención de las 5 combinaciones de hiperparámetros que han devuelto mejor accuracy en la validación  
    df_history10.sort_values(by = 'val_accuracy', ascending = False).head(10)

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Recuperación del dataframe con los resultados del entramiento con los hiperparámetros que han devuelto mejor resultados en la validación
    df_histfit10 = pd.read_pickle(output_path + 'df_histfit10')
    
    # Representación de las gráficas con el accuracy y loss del mejor modelo obtenido
    plot_acc_loss(df_histfit10)

In [ ]:
if test_model_InceptionResNetV2_flag == True:
    # Evaluación del modelo
    if CF1_flag or CF4_flag:
        model = build_model_InceptionResNetV2()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_InceptionResNetV2_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'InceptionResNetV2_best_model.h5')
    model.save(output_path + 'model10.h5') # guardado del modelo completo

    loss, acc = model.evaluate([x_test,x_test_ext], y_test, verbose = 2) # Evaluación con el conjunto de test

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Carga del modelo completo
    if CF1_flag or CF4_flag:
        model = build_model_InceptionResNetV2()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    if CF2_flag or CF3_flag:
        model = build_model_InceptionResNetV2_binary()
        model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.load_weights(output_path + 'model10.h5')

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Predicción con el conjunto de test
    y_pred = model.predict([x_test,x_test_ext])

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Curva ROC multiclase
    plot_multiclass_roc(model, [x_test,x_test_ext], y_test, classes, len(classes), figsize = (14, 7))

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred.argmax(axis = 1))
    
    # Cálculo de la matriz de confusión y representación normalizada y sin normalizar
    plot_confusion_matrix(cm, cm_plot_classes, normalize = True)
    plot_confusion_matrix(cm, cm_plot_classes, normalize = False)

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Métricas
    print('***** MÉTRICAS *****')
    print('Accuracy: {:.5f}'.format(metrics.accuracy_score(y_test, y_pred.argmax(axis = 1))))
    print('Precision: {:.5f}'.format(metrics.precision_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('Recall: {:.5f}'.format(metrics.recall_score(y_test, y_pred.argmax(axis = 1), average ='macro')))
    print('F1 Score: {:.5f}'.format(metrics.f1_score(y_test, y_pred.argmax(axis = 1), average ='macro')))

In [ ]:
if train_model_InceptionResNetV2_flag == True:
    # Dataframe temporal para guardar los accuracies de cada clase de este modelo
    if CF1_flag or CF4_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6'], dtype = np.float64) 
    if CF2_flag or CF3_flag:
        m_temp = pd.DataFrame(columns = ['C0', 'C1'], dtype = np.float64)
    for i in range(len(classes)):
        m_temp['C'+str(i)] = [round(cm[i][i]/np.sum(cm[i]),5)]
        
    # Adición de la fila con el accuracy de cada clase de este modelo a la matriz con el resto de modelos
    m1 = pd.concat([m1, m_temp])

In [ ]:
# Reinicio de los índices
m1 = m1.reset_index()
# Borrado de la columna index
m1 = m1.drop(['index'], axis = 1)

In [ ]:
if save_matriz_pesos_flag == True:
    # Guardado de la matriz de pesos de los modelos
    m1.to_pickle(output_path + 'm1')